# Transcriptomic analysis of SmLE-PZQ-ER and ES


## Aim

Our genome-wide association study (GWAS) revealed that a quantitative trait locus (QTL) on chromosome 3 of *S. mansoni* is linked to praziquantel (PZQ) response. This QTL includes the *Sm.TRPM$_{PZQ}$* gene which is strongly suspected to be the target of PZQ. However no obvious mutations in this gene could explain the resistance. Alternatively, gene expression could be another mechanism, especially knowing that juveniles worms of sensitive populations are naturally resistant. Following our GWAS, we generated two schistosome populations using a marker-assisted selection: SmLE-PZQ-ER population enriched in resistant (ER) allele and SmLE-PZQ-ES encriched in sensitive (ES) allele. The two purified populations differ by more than 377-fold in PZQ response. These populations are therefore ideal to investigate gene regulation that could be linked to PZQ resistance.

To understand this differencein PZQ response, we analyze gene expression of juvenile and adult worms from SmLE-PZQ-ER and SmLE-PZQ-ES populations to test if gene expression differences could be associated to the phenotype. We hypothesize that gene expression should differ between ER and ES adults but not necessarily between ER and ES juveniles as those are naturally recovering from the PZQ treatment (like all juveniles).


## Environment and data

Creating a conda environment improves reproducibility by installing specific versions of the programs used.

In [ ]:
conda env create -f .env/env.yml

The cell below must be run each time a new Jupyter session is run.

In [ ]:
# Activate the environment
source $(sed "s,/bin/conda,," <<<$CONDA_EXE)/etc/profile.d/conda.sh
conda activate PZQ-R_RNA

# Remove potential variable interferences
export PERL5LIB=""
export PYTHONNOUSERSITE=1

The cell below must be run only once at the time of the environment creation.

In [ ]:
# Installing needed R packages
Rscript ".env/R package dependencies.R"

### Sequencing data

This step downloads the fastq files of the different samples from the SRA repository.

In [ ]:
# Data directory
ldir="data/libraries"
[[ ! -d "$ldir" ]] && mkdir -p "$ldir"

In [ ]:
# Bioproject
bioproject=PRJNA704646

# Download related information to data project
wget -q -O runinfo "http://trace.ncbi.nlm.nih.gov/Traces/sra/sra.cgi?save=efetch&rettype=runinfo&db=sra&term=${bioproject}"

# Field of interest (library name and weblink)
fdn=$(head -n 1 runinfo | tr "," "\n" | grep -w -n "SampleName" | cut -d ":" -f 1)
fdr=$(head -n 1 runinfo | tr "," "\n" | grep -w -n "Run" | cut -d ":" -f 1)
flk=$(head -n 1 runinfo | tr "," "\n" | grep -w -n "download_path" | cut -d ":" -f 1)

# Download fastq files
while read line
do
    # Filename, run and download link
    fln=$(cut -d "," -f $fdn <<<$line)
    run=$(cut -d "," -f $fdr <<<$line)
    lnk=$(cut -d "," -f $flk <<<$line)
    
    # Download
    echo "$fln"
    [[ ! -d "$ldir/$fln/" ]] && mkdir -p "$ldir/$fln/"
    retry=0
    
    while [[ $retry -lt 2 ]]
    do
        # Download sra file
        wget -q -c -O "$ldir/$fln/$run" "$lnk"
        # Check integrity
        vdb-validate -q "$ldir/$fln/$run" &> /dev/null
        [[ $? -ne 0 ]] && ((retry++)) || break
    done
    
    # If max download attempt reached, issue message and move to the next
    [[ $retry -eq 2 ]] && echo "$run: dowloading problem" >> "$ldir/download_issue" && contine
    
    # Convert sra into fastq
    fastq-dump -O "$ldir/$fln/" --split-files "$ldir/$fln/$run"
    rm "$ldir/$fln/$run"
    
    # Rename file with more meaningful name
    mv "$ldir/$fln/${run}_1.fastq" "$ldir/$fln/${fln}_R1.fastq"
    mv "$ldir/$fln/${run}_2.fastq" "$ldir/$fln/${fln}_R2.fastq"
    
done < <(tail -n +2 runinfo | sed "/^$/d")

# Compress files
pigz "$ldir/"*/*

rm runinfo*

### Genome data and annotation

The genome data and its annotation is downloaded from the [WormBase ParaSite](https://parasite.wormbase.org). We use the data from the version 14 (WBPS14).

In [ ]:
gdir="data/genome"
[[ ! -d "$gdir" ]] && mkdir "$gdir"

# Genome
wget -P "$gdir" ftp://ftp.ebi.ac.uk/pub/databases/wormbase/parasite/releases/WBPS14/species/schistosoma_mansoni/PRJEA36577/schistosoma_mansoni.PRJEA36577.WBPS14.genomic.fa.gz
pigz -d "$gdir/schistosoma_mansoni.PRJEA36577.WBPS14.genomic.fa.gz"

# Annotation
wget -P "$gdir" ftp://ftp.ebi.ac.uk/pub/databases/wormbase/parasite/releases/WBPS14/species/schistosoma_mansoni/PRJEA36577/schistosoma_mansoni.PRJEA36577.WBPS14.annotations.gff3.gz
pigz -d "$gdir/schistosoma_mansoni.PRJEA36577.WBPS14.annotations.gff3.gz"

STAR will be used to align data and RSEM will be used to generate transcript per million (TPM) counts. These two tools need to pre-process the reference genome.

### STAR reference genome

Creating a STAR reference genome requires the use of an annotation file. A GFF file is a format that can be normally used with STAR. However my first attempt to generate a STAR reference genome using the official GFF file of the genome and the `--sjdbGTFtagExonParentTranscript Parent` option as mentioned in the manual did not allow me to get gene counts after running STAR on sample (the gene count file contains the first 4 lines only). This problem is very similar to [this](https://groups.google.com/forum/#!msg/rna-star/oRvzihFXE8k/Xa-7YgUUBgAJ). Therefore I converted the GFF file into a GTF file which is the default format used by STAR and this solved the problem.

In [ ]:
# Convert GFF into GTF file
gffread "$gdir/schistosoma_mansoni.PRJEA36577.WBPS14.annotations.gff3" -T -o "$gdir/schistosoma_mansoni.PRJEA36577.WBPS14.annotations.gtf"

In [ ]:
# Overhang
i=149

# Make STAR ref folder
mkdir "$gdir/S.mansoni_STAR_${i}"

STAR --runMode genomeGenerate \
     --runThreadN $(nproc)    \
     --genomeDir "$gdir/S.mansoni_STAR_${i}" \
     --genomeFastaFiles "$gdir/schistosoma_mansoni.PRJEA36577.WBPS14.genomic.fa" \
     --sjdbGTFfile "$gdir/schistosoma_mansoni.PRJEA36577.WBPS14.annotations.gtf" \
     --sjdbOverhang $i

# Move log
mv Log.out "$gdir/S.mansoni_STAR_${i}/"

### RSEM reference

RSEM requires to generate a reference using the GFF or GTF and the reference genome file as mentioned in the documentation.

In [ ]:
# Make RSEM ref folder
mkdir "$gdir/S.mansoni_RSEM"

rsem-prepare-reference --gtf "$gdir/schistosoma_mansoni.PRJEA36577.WBPS14.annotations.gtf" \
        -p $(nproc) \
        "$gdir/schistosoma_mansoni.PRJEA36577.WBPS14.genomic.fa" \
        "$gdir/S.mansoni_RSEM/S.mansoni" > "$gdir/S.mansoni_RSEM/log"

## Data quality

To ensure that sequencing was done properly, we analyze the data quality of each library using MultiQC.

In [ ]:
# Directory to store status files
statdir="status"
[[ ! -d "$statdir" ]] && mkdir "$statdir"

# Directory to store results
resdir="results"
[[ -d "$resdir" ]] || mkdir "$resdir"

In [ ]:
# Run FastQC for each fastq file
for i in $(find . -name *fastq.gz)
do
    qsub -V -cwd -o "$statdir" -j y -r y -pe smp 12 -b y fastqc --noextract -t $(nproc) "$i"
done

# Global MultiQC report
multiqc -ip -o "$resdir/1-report/" data/libraries

Report is located in `results/1-report`.

## Alignment and quantification

We use a snakemake pipeline to align using STAR and perform transcript quantification using RSEM. The command used is written to run the pipeline on Grid Engine cluster. If data and genome folder have been modified, these modifications must be reported in the snakemake file.

In [ ]:
# Directory to store status files
statdir=status
[[ ! -d "$statdir" ]] && mkdir "$statdir"

# Snakemake pipeline
snakemake --snakefile snakefile --cluster "qsub -V -cwd -o "$statdir" -j y -r y -pe smp 10 -S /bin/bash" --jobs 24 -w 120

After alignment, gene and isoforms are counted using RSEM.

In [ ]:
# Quuantification folder
qdir="$resdir/3-quantification"
[[ ! -d "$qdir" ]] && mkdir "$qdir"

rsem-generate-data-matrix "$ldir/"*/*isoforms.results > "$qdir/PZQ-ER-ES.isoform.counts.matrix"
rsem-generate-data-matrix "$ldir/"*/*genes.results > "$qdir/PZQ-ER-ES.gene.counts.matrix"

## Gene and isoform expression analysis


### Factors structuring the data

We first ensure that the transcriptomic variations is first explained by the sample type rather than any other factors. For this we performed a GLM-PCA on the expression data of each sample.

In [ ]:
# GLM-PCA to test if other than biological effects exists
Rscript scripts/RNA-seq_PCA.R

This shows that the samples clustered first by stage, then by sex. No other factors (like sequencing lane) explain the structuration of the data.


### Global analysis

We look at differences in gene and isoform expression between ER and ES populations and between stages. We also highlight the genes under QTL 2 and QTL 3 (listed in `results/2-QTL`). The QTL were obtained from the genome-wide association study (GWAS) conducted on SmLE-PZQ-R (see the [GWAS analysis](https://github.com/fdchevalier/PZQ-R_DNA-seq)).

In [ ]:
# Directory
genedir="results/2-QTL"
[[ ! -d "$genedir" ]] && mkdir -p "$genedir"

# BED files of the QTL boundaries
bed2=$(echo -e "SM_V7_2\t291191\t1457462")
bed3=$(echo -e "SM_V7_3\t22805\t4013538")

# List of genes under QTL of chr. 2
bedtools intersect -a "$gdir/schistosoma_mansoni.PRJEA36577.WBPS14.annotations.gff3" -b <(echo "$bed2") -wa |\
    awk '$3 == "gene"' |\
    cut -f 9 |\
    cut -d ";" -f 1 |\
    cut -d ":" -f 2 |\
    sort -u > "$genedir/QTL_genes_chr2"

#List of genes under QTL of chr. 3
bedtools intersect -a "$gdir/schistosoma_mansoni.PRJEA36577.WBPS14.annotations.gff3" -b <(echo "$bed3") -wa |\
    awk '$3 == "gene"' |\
    cut -f 9 |\
    cut -d ";" -f 1 |\
    cut -d ":" -f 2 |\
    sort -u > "$genedir/QTL_genes_chr3"

In [ ]:
# Formal analysis of the complete RNA-seq data
Rscript scripts/RNA-seq_analysis.R

Results show that gene expression of *Sm.TRPM$_{PZQ}$* is lower in ER males than ES males and this is mainly driven by one isoform (isoform 6).


### Sm.TRPM$_{PZQ}$ expression analysis

We focus our analysis on the *Sm.TRPM$_{PZQ}$* gene specifically to understand pattern of expression in each schistosome stage and sex.

In [ ]:
# Specific analysis of Smp_246790
Rscript scripts/RNA-seq_TRP_analysis.R

Results show that gene expression of *Sm.TRPM$_{PZQ}$* in male or female juveniles is similar to the expression of adult males. However, the isoform 6 is lower expressed in ER and ES juveniles and ER males but not ES males which could explained the phenotype observed.

Scripts generating the figures included in the manuscript.

In [ ]:
Rscript scripts/Fig5.R
Rscript scripts/Supp_Fig5.R